In [6]:
class Graph:
    def __init__(self):
        # 인접 리스트로 그래프 표현
        self.graph = {}

    def add_edge(self, u, v):
        if u not in self.graph:
            self.graph[u] = []
        self.graph[u].append(v)
        print(self.graph)

    def dfs_with_explanation(self, start):
        stack = [start]  # 시작 노드를 스택에 넣음
        visited = set()  # 방문한 노드를 추적

        print(f"\n==== DFS 탐색 시작 (시작 노드: {start}) ====")

        while stack:
            # 현재 스택과 방문 상태 출력
            print(f"\n현재 스택: {stack}")
            print(f"방문한 노드: {visited}")

            # 스택에서 노드를 꺼냄 (가장 최근에 추가된 노드)
            current = stack.pop()
            print(f"\n노드 {current} 처리 중…")

            if current not in visited:
                print(f"노드 {current} 방문 처리")
                visited.add(current)

                # 인접한 노드들을 스택에 추가
                neighbors = self.graph.get(current, [])
                print(
                    f"노드 {current}의 미방문 이웃 노드들을 스택에 추가: {[n for n in neighbors if n not in visited]}"
                )

                # 이웃 노드들을 스택에 추가 (역순으로 추가하여 원하는 순서로 방문)
                for neighbor in reversed(neighbors):
                    if neighbor not in visited:
                        stack.append(neighbor)

        return visited

In [ ]:
# 예제 사용
graph = Graph()
# 아래와 같은 그래프 구성
# 1 -- 2 -- 4
# |    |
# 3    5
graph.add_edge(1, 2)
graph.add_edge(1, 3)
graph.add_edge(2, 4)
graph.add_edge(2, 5)

In [ ]:
visited = graph.dfs_with_explanation(1)
print("\n==== DFS 탐색 완료 ====")
print(f"최종 방문 순서: {visited}")


In [ ]:
import random
from dataclasses import dataclass
from typing import Any, Optional


@dataclass
class Packet:
    """네트워크 패킷을 표현하는 클래스"""

    data: Any
    source_address: Optional[str] = None
    destination_address: Optional[str] = None
    protocol: Optional[str] = None
    port: Optional[int] = None
    sequence_number: Optional[int] = None
    frame_check_sequence: Optional[str] = None


class OSILayer:
    """OSI 계층의 기본 클래스"""

    def __init__(self, name: str):
        self.name = name

    def process_down(self, packet: Packet) -> Packet:
        """데이터를 하위 계층으로 전달하는 과정 (캡슐화)"""
        print(f"[{self.name}] 계층 캡슐화 처리")
        return packet

    def process_up(self, packet: Packet) -> Packet:
        """데이터를 상위 계층으로 전달하는 과정 (디캡슐화)"""
        print(f"[{self.name}] 계층 디캡슐화 처리")
        return packet


class ApplicationLayer(OSILayer):
    """7계층 - 응용 계층
    사용자와 네트워크 간의 인터페이스 제공
    HTTP, FTP, SMTP 등의 프로토콜 구현
    """

    def __init__(self):
        super().__init__("Application")

    def process_down(self, data: str) -> Packet:
        print(f"[{self.name}] HTTP 요청 생성")
        return Packet(data=data, protocol="HTTP")

    def process_up(self, packet: Packet) -> str:
        print(f"[{self.name}] 응용 프로그램에 데이터 전달: {packet.data}")
        return packet.data


class PresentationLayer(OSILayer):
    """6계층 - 표현 계층
    데이터 형식 변환, 암호화/복호화, 인코딩/디코딩
    """

    def __init__(self):
        super().__init__("Presentation")

    def process_down(self, packet: Packet) -> Packet:
        print(f"[{self.name}] 데이터 인코딩 및 암호화")
        # 간단한 암호화 시뮬레이션 (실제로는 더 복잡한 암호화 사용)
        packet.data = f"ENCRYPTED({packet.data})"
        return packet

    def process_up(self, packet: Packet) -> Packet:
        print(f"[{self.name}] 데이터 디코딩 및 복호화")
        # 암호화된 데이터 복호화
        packet.data = packet.data.replace("ENCRYPTED(", "").replace(")", "")
        return packet


class SessionLayer(OSILayer):
    """5계층 - 세션 계층
    통신 세션 설정, 유지, 종료
    동기화 및 대화 제어
    """

    def __init__(self):
        super().__init__("Session")

    def process_down(self, packet: Packet) -> Packet:
        print(f"[{self.name}] 세션 설정 및 관리")
        packet.sequence_number = random.randint(1000, 9999)
        return packet

    def process_up(self, packet: Packet) -> Packet:
        print(f"[{self.name}] 세션 확인 (시퀀스 번호: {packet.sequence_number})")
        return packet


class TransportLayer(OSILayer):
    """4계층 - 전송 계층
    종단 간 통신 제어
    TCP/UDP 프로토콜, 포트 할당
    """

    def __init__(self):
        super().__init__("Transport")

    def process_down(self, packet: Packet) -> Packet:
        print(f"[{self.name}] TCP 세그먼트 생성")
        packet.port = 80  # HTTP 기본 포트
        return packet

    def process_up(self, packet: Packet) -> Packet:
        print(f"[{self.name}] 포트 확인 및 세그먼트 재조립")
        return packet


class NetworkLayer(OSILayer):
    """3계층 - 네트워크 계층
    경로 설정 및 논리 주소 지정
    IP 프로토콜 처리
    """

    def __init__(self):
        super().__init__("Network")

    def process_down(self, packet: Packet) -> Packet:
        print(f"[{self.name}] IP 패킷 생성")
        packet.source_address = "192.168.1.1"
        packet.destination_address = "192.168.1.2"
        return packet

    def process_up(self, packet: Packet) -> Packet:
        print(f"[{self.name}] 목적지 주소 확인: {packet.destination_address}")
        return packet


class DataLinkLayer(OSILayer):
    """2계층 - 데이터링크 계층
    물리 주소 지정 (MAC)
    프레임 생성 및 오류 검출
    """

    def __init__(self):
        super().__init__("DataLink")

    def process_down(self, packet: Packet) -> Packet:
        print(f"[{self.name}] 프레임 생성 및 MAC 주소 할당")
        # CRC 체크섬 생성 시뮬레이션
        packet.frame_check_sequence = "CRC32"
        return packet

    def process_up(self, packet: Packet) -> Packet:
        print(f"[{self.name}] 프레임 무결성 검사 (FCS: {packet.frame_check_sequence})")
        return packet


class PhysicalLayer(OSILayer):
    """1계층 - 물리 계층
    비트 단위 데이터 전송
    전기 신호 변환
    """

    def __init__(self):
        super().__init__("Physical")

    def process_down(self, packet: Packet) -> str:
        print(f"[{self.name}] 비트스트림으로 변환하여 전송")
        # 실제 비트스트림 대신 문자열로 시뮬레이션
        return f"BITSTREAM({packet.data})"

    def process_up(self, bitstream: str) -> Packet:
        print(f"[{self.name}] 비트스트림을 데이터로 변환")
        # 비트스트림에서 원래 데이터 추출
        data = bitstream.replace("BITSTREAM(", "").replace(")", "")
        return Packet(data=data)


class NetworkSimulator:
    """OSI 7계층 네트워크 통신 시뮬레이터"""

    def __init__(self):
        # 각 계층 초기화
        self.application = ApplicationLayer()
        self.presentation = PresentationLayer()
        self.session = SessionLayer()
        self.transport = TransportLayer()
        self.network = NetworkLayer()
        self.datalink = DataLinkLayer()
        self.physical = PhysicalLayer()

    def send_data(self, data: str):
        """데이터 송신 프로세스 (캡슐화)"""
        print("\n=== 송신 프로세스 시작 (캡슐화) ===")

        # 응용 계층부터 물리 계층까지 차례로 처리
        packet = self.application.process_down(data)
        packet = self.presentation.process_down(packet)
        packet = self.session.process_down(packet)
        packet = self.transport.process_down(packet)
        packet = self.network.process_down(packet)
        packet = self.datalink.process_down(packet)
        bitstream = self.physical.process_down(packet)

        print("\n전송된 비트스트림:", bitstream)
        return bitstream

    def receive_data(self, bitstream: str) -> str:
        """데이터 수신 프로세스 (디캡슐화)"""
        print("\n=== 수신 프로세스 시작 (디캡슐화) ===")

        # 물리 계층부터 응용 계층까지 차례로 처리
        packet = self.physical.process_up(bitstream)
        packet = self.datalink.process_up(packet)
        packet = self.network.process_up(packet)
        packet = self.transport.process_up(packet)
        packet = self.session.process_up(packet)
        packet = self.presentation.process_up(packet)
        data = self.application.process_up(packet)

        return data


# 사용 예시
if __name__ == "__main__":
    # 네트워크 시뮬레이터 인스턴스 생성
    simulator = NetworkSimulator()

    # 테스트 데이터 전송
    original_data = "Hello, OSI World!"
    print(f"\n원본 데이터: {original_data}")

    # 데이터 송신
    bitstream = simulator.send_data(original_data)

    # 데이터 수신
    received_data = simulator.receive_data(bitstream)
    print(f"\n수신된 데이터: {received_data}")

In [ ]:
import json
from dataclasses import dataclass
from typing import Any, Dict, Optional


# 각 계층별 헤더/트레일러 정보를 담는 클래스들
@dataclass
class ApplicationHeader:
    content_type: str
    encoding: str


@dataclass
class PresentationHeader:
    encryption: str
    compression: str


@dataclass
class SessionHeader:
    session_id: str
    sequence_number: int


@dataclass
class TransportHeader:
    source_port: int
    destination_port: int
    sequence_number: int
    checksum: str


@dataclass
class NetworkHeader:
    source_ip: str
    destination_ip: str
    protocol: str
    ttl: int


@dataclass
class DataLinkHeader:
    source_mac: str
    destination_mac: str
    frame_type: str


@dataclass
class DataLinkTrailer:
    frame_check_sequence: str


class OSILayer:
    """OSI 계층의 기본 클래스"""

    def __init__(self, name: str):
        self.name = name

    def encapsulate(self, data: Any) -> Dict:
        """데이터 캡슐화 (추상 메서드)"""
        raise NotImplementedError

    def decapsulate(self, data: Dict) -> Any:
        """데이터 디캡슐화 (추상 메서드)"""
        raise NotImplementedError


class ApplicationLayer(OSILayer):
    """7계층 - 응용 계층
    사용자와 가장 가까운 계층으로 응용 프로그램 간의 데이터 교환을 담당"""

    def __init__(self):
        super().__init__("Application Layer")

    def encapsulate(self, data: str) -> Dict:
        print(f"[{self.name}] Encapsulating data...")
        header = ApplicationHeader(content_type="text/plain", encoding="utf-8")
        return {"header": header, "data": data}

    def decapsulate(self, data: Dict) -> str:
        print(f"[{self.name}] Decapsulating data...")
        return data["data"]


class PresentationLayer(OSILayer):
    """6계층 - 표현 계층
    데이터의 형식 변환, 암호화, 압축을 담당"""

    def __init__(self):
        super().__init__("Presentation Layer")

    def encapsulate(self, data: Dict) -> Dict:
        print(f"[{self.name}] Encapsulating data...")
        header = PresentationHeader(encryption="AES-256", compression="gzip")
        return {"header": header, "data": data}

    def decapsulate(self, data: Dict) -> Dict:
        print(f"[{self.name}] Decapsulating data...")
        return data["data"]


class SessionLayer(OSILayer):
    """5계층 - 세션 계층
    통신 세션 구축 및 관리를 담당"""

    def __init__(self):
        super().__init__("Session Layer")
        self._session_counter = 0

    def encapsulate(self, data: Dict) -> Dict:
        print(f"[{self.name}] Encapsulating data...")
        self._session_counter += 1
        header = SessionHeader(
            session_id=f"SESSION_{self._session_counter}", sequence_number=1
        )
        return {"header": header, "data": data}

    def decapsulate(self, data: Dict) -> Dict:
        print(f"[{self.name}] Decapsulating data...")
        return data["data"]


class TransportLayer(OSILayer):
    """4계층 - 전송 계층
    종단간 신뢰성 있는 데이터 전송을 담당"""

    def __init__(self):
        super().__init__("Transport Layer")

    def encapsulate(self, data: Dict) -> Dict:
        print(f"[{self.name}] Encapsulating data...")
        header = TransportHeader(
            source_port=12345, destination_port=80, sequence_number=1, checksum="ABC123"
        )
        return {"header": header, "data": data}

    def decapsulate(self, data: Dict) -> Dict:
        print(f"[{self.name}] Decapsulating data...")
        return data["data"]


class NetworkLayer(OSILayer):
    """3계층 - 네트워크 계층
    패킷의 경로 설정과 주소 지정을 담당"""

    def __init__(self):
        super().__init__("Network Layer")

    def encapsulate(self, data: Dict) -> Dict:
        print(f"[{self.name}] Encapsulating data...")
        header = NetworkHeader(
            source_ip="192.168.1.1",
            destination_ip="192.168.1.2",
            protocol="TCP",
            ttl=64,
        )
        return {"header": header, "data": data}

    def decapsulate(self, data: Dict) -> Dict:
        print(f"[{self.name}] Decapsulating data...")
        return data["data"]


class DataLinkLayer(OSILayer):
    """2계층 - 데이터 링크 계층
    물리적 주소 지정과 오류 검출을 담당"""

    def __init__(self):
        super().__init__("Data Link Layer")

    def encapsulate(self, data: Dict) -> Dict:
        print(f"[{self.name}] Encapsulating data...")
        header = DataLinkHeader(
            source_mac="00:11:22:33:44:55",
            destination_mac="66:77:88:99:AA:BB",
            frame_type="Ethernet II",
        )
        trailer = DataLinkTrailer(frame_check_sequence="CRC32_XYZ")
        return {"header": header, "data": data, "trailer": trailer}

    def decapsulate(self, data: Dict) -> Dict:
        print(f"[{self.name}] Decapsulating data...")
        return data["data"]


class PhysicalLayer(OSILayer):
    """1계층 - 물리 계층
    비트 스트림의 전송을 담당"""

    def __init__(self):
        super().__init__("Physical Layer")

    def encapsulate(self, data: Dict) -> str:
        print(f"[{self.name}] Converting to bit stream...")
        # 실제 비트 스트림 변환 대신 JSON 문자열로 시뮬레이션
        return json.dumps(data, default=lambda x: x.__dict__)

    def decapsulate(self, data: str) -> Dict:
        print(f"[{self.name}] Converting from bit stream...")
        # JSON 문자열에서 딕셔너리로 변환
        return json.loads(data)


class NetworkStack:
    """전체 OSI 7계층을 관리하는 클래스"""

    def __init__(self):
        self.application = ApplicationLayer()
        self.presentation = PresentationLayer()
        self.session = SessionLayer()
        self.transport = TransportLayer()
        self.network = NetworkLayer()
        self.datalink = DataLinkLayer()
        self.physical = PhysicalLayer()

    def send_data(self, data: str):
        """데이터 송신 - 캡슐화 과정"""
        print("\n=== Starting Data Transmission ===")

        # 상위 계층에서 하위 계층으로 캡슐화
        encapsulated_data = self.application.encapsulate(data)
        encapsulated_data = self.presentation.encapsulate(encapsulated_data)
        encapsulated_data = self.session.encapsulate(encapsulated_data)
        encapsulated_data = self.transport.encapsulate(encapsulated_data)
        encapsulated_data = self.network.encapsulate(encapsulated_data)
        encapsulated_data = self.datalink.encapsulate(encapsulated_data)
        bit_stream = self.physical.encapsulate(encapsulated_data)

        return bit_stream

    def receive_data(self, bit_stream: str):
        """데이터 수신 - 디캡슐화 과정"""
        print("\n=== Starting Data Reception ===")

        # 하위 계층에서 상위 계층으로 디캡슐화
        decapsulated_data = self.physical.decapsulate(bit_stream)
        decapsulated_data = self.datalink.decapsulate(decapsulated_data)
        decapsulated_data = self.network.decapsulate(decapsulated_data)
        decapsulated_data = self.transport.decapsulate(decapsulated_data)
        decapsulated_data = self.session.decapsulate(decapsulated_data)
        decapsulated_data = self.presentation.decapsulate(decapsulated_data)
        original_data = self.application.decapsulate(decapsulated_data)

        return original_data


# 사용 예시
def main():
    # 네트워크 스택 초기화
    network = NetworkStack()

    # 전송할 데이터
    original_message = "Hello, OSI World!"
    print(f"\nOriginal Message: {original_message}")

    # 데이터 송신 (캡슐화)
    transmitted_data = network.send_data(original_message)
    print(f"\nTransmitted Data (bit stream):\n{transmitted_data}")

    # 데이터 수신 (디캡슐화)
    received_message = network.receive_data(transmitted_data)
    print(f"\nReceived Message: {received_message}")


if __name__ == "__main__":
    main()


In [ ]:
def solution(s):
    """
    이진 변환 횟수와 제거된 0의 개수를 계산하는 함수

    Args:
        s (str): 0과 1로 이루어진 이진수 문자열

    Returns:
        list: [이진 변환 횟수, 제거된 총 0의 개수]
    """
    transform_count = 0  # 이진 변환 횟수
    removed_zeros = 0  # 제거된 0의 총 개수

    while s != "1":
        # 현재 문자열의 0 개수를 세고, 제거된 0의 개수에 추가
        zeros_count = s.count("0")
        removed_zeros += zeros_count

        # 0을 제거하고 남은 1의 개수를 계산
        ones_count = len(s) - zeros_count

        # 남은 1의 개수를 2진수 문자열로 변환
        # bin() 함수는 '0b'를 접두어로 추가하므로 [2:]로 제거
        s = bin(ones_count)[2:]

        transform_count += 1

    return [transform_count, removed_zeros]


# 테스트
test_cases = ["110010101001", "01110", "1111111"]
for test in test_cases:
    result = solution(test)
    print(f"입력: {test}")
    print(f"결과: {result}\n")


In [ ]:
def greedy_set_cover(universe, subsets):
    uncovered = set(universe)
    cover = []

    while uncovered:
        best_subset = max(subsets, key=lambda s: len(uncovered & s))
        cover.append(best_subset)
        uncovered -= best_subset
        subsets.remove(best_subset)

    return cover


# 예시 사용
universe = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
subsets = [
    {1, 2, 3, 8},
    {1, 2, 3, 4, 8},
    {1, 2, 3, 4},
    {2, 3, 4, 5, 7, 8},
    {4, 5, 6, 7},
    {5, 6, 7, 9, 10},
    {4, 5, 6, 7},
    {1, 2, 4, 8},
    {6, 9},
    {6, 10},
]

cover = greedy_set_cover(universe, subsets)
print("Approximate Set Cover:", cover)


In [ ]:
# 탐욕적 방법으로 집합 커버 문제를 해결하는 함수
# universe: 전체 집합 (모든 마을들)
# subsets: 각 학교 위치별로 15분 내 통학 가능한 마을들의 집합을 담은 리스트
def greedy_set_cover(universe, subsets):
    # 아직 학교가 배정되지 않은 마을들을 추적하기 위한 집합
    # 처음에는 모든 마을이 미배정 상태
    uncovered = set(universe)
    # 선택된 학교 위치들(부분집합들)을 저장할 리스트
    cover = []

    # 모든 마을이 학교에 배정될 때까지 반복
    while uncovered:
        # 현재 미배정된 마을들 중에서 가장 많은 마을을 커버할 수 있는 학교 위치 선택
        # key 함수는 각 부분집합과 미배정 마을들의 교집합 크기를 반환
        # 즉, 해당 위치에 학교를 지었을 때 새롭게 커버되는 마을의 수를 계산
        best_subset = max(subsets, key=lambda s: len(uncovered & s))
        print("bs", best_subset)
        # 선택된 학교 위치를 결과 리스트에 추가
        cover.append(best_subset)
        print("cover", cover)

        # 선택된 학교가 커버하는 마을들을 미배정 집합에서 제거
        # -= 연산자는 차집합 연산을 수행
        uncovered -= best_subset
        print("uncovered", uncovered)

        # 이미 선택된 학교 위치는 다시 선택되지 않도록 후보 리스트에서 제거
        subsets.remove(best_subset)
        print("subsets", subsets)

    # 선택된 모든 학교 위치들의 리스트 반환
    return cover


# 테스트를 위한 입력 데이터 설정
# universe: 1부터 10까지의 모든 마을 번호
universe = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

# subsets: 각 위치에 학교를 지었을 때 15분 내 통학 가능한 마을들의 집합
# 예: 첫 번째 집합 {1, 2, 3, 8}은 1번 위치에 학교를 지으면
# 1, 2, 3, 8번 마을의 학생들이 15분 내 통학 가능함을 의미
subsets = [
    {1, 2, 3, 8},  # 1번 위치의 학교가 커버하는 마을들
    {1, 2, 3, 4, 8},  # 2번 위치의 학교가 커버하는 마을들
    {1, 2, 3, 4},  # 3번 위치의 학교가 커버하는 마을들
    {2, 3, 4, 5, 7, 8},  # 4번 위치의 학교가 커버하는 마을들
    {4, 5, 6, 7},  # 5번 위치의 학교가 커버하는 마을들
    {5, 6, 7, 9, 10},  # 6번 위치의 학교가 커버하는 마을들
    {4, 5, 6, 7},  # 7번 위치의 학교가 커버하는 마을들
    {1, 2, 4, 8},  # 8번 위치의 학교가 커버하는 마을들
    {6, 9},  # 9번 위치의 학교가 커버하는 마을들
    {6, 10},  # 10번 위치의 학교가 커버하는 마을들
]

# 알고리즘 실행 및 결과 출력
cover = greedy_set_cover(universe, subsets)
print("Approximate Set Cover:", cover)


In [ ]:
class Item:
    def __init__(self, weight, value, index):
        self.weight = weight  # 물건의 무게
        self.value = value  # 물건의 가치
        self.index = index  # 물건의 인덱스
        self.ratio = value / weight  # 단위 무게당 가치


def knapsack_branch_and_bound(items, capacity):
    n = len(items)

    # 단위 무게당 가치 기준으로 정렬 (한정 함수의 효율을 위해)
    items.sort(key=lambda x: x.ratio, reverse=True)

    # 최적해를 저장할 변수들
    max_value = 0  # 지금까지 찾은 최적해의 가치
    best_solution = [0] * n  # 최적해의 선택 여부 저장

    def bound(node_level, current_weight, current_value):
        """
        현재 노드에서 가능한 최대 가치를 계산하는 한정 함수
        이 값이 현재까지의 최적해보다 작다면 더 탐색할 필요가 없음
        """
        if current_weight >= capacity:
            return 0

        # 현재 노드의 한계값 계산
        bound_value = current_value
        total_weight = current_weight
        j = node_level

        # 남은 물건들을 가치/무게 비율이 높은 순서대로 분할하여 추가
        while j < n and total_weight + items[j].weight <= capacity:
            total_weight += items[j].weight
            bound_value += items[j].value
            j += 1

        # 마지막 물건은 분할하여 추가
        if j < n:
            bound_value += (capacity - total_weight) * items[j].ratio

        return bound_value

    def branch(node_level, current_weight, current_value, solution):
        """
        분기 함수: 각 물건을 선택하거나 선택하지 않는 두 가지 경우로 분기
        """
        nonlocal max_value, best_solution

        # 모든 물건을 고려했다면 현재 해답 평가
        if node_level == n:
            if current_value > max_value:
                max_value = current_value
                best_solution = solution[:]
            return

        # 한계값 계산하여 가지치기 여부 결정
        if bound(node_level, current_weight, current_value) <= max_value:
            return  # 유망하지 않은 노드는 가지치기

        # 현재 물건을 선택하는 경우 탐색
        if current_weight + items[node_level].weight <= capacity:
            solution[items[node_level].index] = 1
            branch(
                node_level + 1,
                current_weight + items[node_level].weight,
                current_value + items[node_level].value,
                solution,
            )

        # 현재 물건을 선택하지 않는 경우 탐색
        solution[items[node_level].index] = 0
        branch(node_level + 1, current_weight, current_value, solution)

    # 초기 호출
    branch(0, 0, 0, [0] * n)
    return max_value, best_solution


# 사용 예시
if __name__ == "__main__":
    # 테스트 데이터
    weights = [2, 3.14, 1.98, 5, 3]  # 물건들의 무게
    values = [40, 50, 100, 95, 30]  # 물건들의 가치
    capacity = 10  # 배낭의 용량

    # Item 객체들 생성
    items = [Item(w, v, i) for i, (w, v) in enumerate(zip(weights, values))]

    # 알고리즘 실행
    max_value, solution = knapsack_branch_and_bound(items, capacity)

    print(f"최대 가치: {max_value}")
    print(f"선택된 물건들: {solution}")


In [ ]:
def fibonacci_dp(n):
    # 메모이제이션을 위한 배열 초기화
    fib = [0] * (n + 1)
    print(fib)
    # 기저 사례 처리
    fib[0] = 0
    fib[1] = 1
    print(fib)
    # 상향식으로 피보나치 수열 계산
    for i in range(2, n + 1):
        # 점화식: F(n) = F(n-1) + F(n-2)
        fib[i] = fib[i - 1] + fib[i - 2]
    print(fib)
    return fib[n]


# 사용 예
n = 10
result = fibonacci_dp(n)
print(f"피보나치 수열의 {n}번째 항: {result}")


In [ ]:
def longest_increasing_subsequence(arr):
    """
    주어진 배열에서 최장 증가 부분 수열의 길이를 찾는 함수

    예시:
    입력: [10, 22, 9, 33, 21, 50, 41, 60, 80]
    출력: 6 (증가 수열: 10 -> 22 -> 33 -> 50 -> 60 -> 80)

    Args:
        arr (list): 정수 배열

    Returns:
        tuple: (최장 증가 부분 수열의 길이, 해당 수열)
    """
    n = len(arr)
    if n == 0:
        return 0, []

    # dp[i]는 arr[i]로 끝나는 최장 증가 부분 수열의 길이
    dp = [1] * n

    # 각 위치에서 이전 요소 중 어떤 것을 선택했는지 추적
    prev = [-1] * n

    # 각 위치에 대해
    for i in range(1, n):
        # 현재 위치 이전의 모든 요소들을 검사
        for j in range(i):
            # 현재 숫자가 이전 숫자보다 크고
            # 이전 숫자로 끝나는 수열에 현재 숫자를 추가했을 때 더 긴 수열이 된다면
            if arr[i] > arr[j] and dp[j] + 1 > dp[i]:
                dp[i] = dp[j] + 1  # 길이 갱신
                prev[i] = j  # 이전 위치 저장

    # 최장 길이와 그 위치 찾기
    max_length = max(dp)
    last_pos = dp.index(max_length)

    # 실제 수열 재구성
    sequence = []
    while last_pos != -1:
        sequence.append(arr[last_pos])
        last_pos = prev[last_pos]

    return max_length, sequence[::-1]  # 수열을 역순으로 반환


# 테스트
def test_lis():
    # 테스트 케이스
    test_cases = [
        [10, 22, 9, 33, 21, 50, 41, 60, 80],
        [3, 10, 2, 1, 20],
        [1, 2, 3, 4, 5],
        [5, 4, 3, 2, 1],
        [],
    ]

    for arr in test_cases:
        length, sequence = longest_increasing_subsequence(arr)
        print(f"배열: {arr}")
        print(f"최장 증가 부분 수열의 길이: {length}")
        print(f"해당 수열: {sequence}")
        print("-" * 50)


# 실행
test_lis()

In [ ]:
def coin_change(amount, coins):
    # 큰 단위의 동전부터 사용하기 위해 내림차순 정렬
    print(coins)
    coins.sort(reverse=True)
    print(coins)
    change = []
    for coin in coins:
        # 현재 동전으로 거슬러 줄 수 있는 만큼 거슬러 줌
        while amount >= coin:
            change.append(coin)
            amount -= coin

    return change


# 사용 예
amount = 63
coins = [25, 10, 5, 1]  # 사용 가능한 동전 단위
result = coin_change(amount, coins)
print(f"{amount}원을 거슬러 주기 위한 동전: {result}")
print(f"사용된 동전 개수: {len(result)}")


In [ ]:
set([1]).intersection(set([2])) == False

In [ ]:
def solution(want, number, discount):
    answer = 0

    for i in range(len(discount)):
        target_range = discount[i:]
        want_number = dict(zip(want, number))
        print(want_number)
        for d in target_range:
            if d not in want:
                break
            else:
                want_number[d] = want_number[d] - 1
                if sum(want_number.values()) == 0:
                    break
        print(target_range, i, want_number)
        print()
        if sum(want_number.values()) == 0:
            answer = i + 1
            break

    return answer


solution(
    ["banana", "apple", "rice", "pork", "pot"],
    [3, 2, 2, 2, 1],
    [
        "chicken",
        "apple",
        "apple",
        "banana",
        "rice",
        "apple",
        "pork",
        "banana",
        "pork",
        "rice",
        "pot",
        "banana",
        "apple",
        "banana",
    ],
)

In [7]:
def solution(m, n, puddles):
    answer = 0
    MOD = 1000000007
    if m == 1 and n == 1:
        raise ValueError
    else:
        if 1 > m or m > 100:
            raise ValueError
        if 1 > n or n > 100:
            raise ValueError

    dp = [[0] * (m + 1) for _ in range(n + 1)]
    print(dp)
    # 시작점 초기화
    dp[1][1] = 1
    print(dp)

    print()
    print(puddles)
    puddles = set((y, x) for x, y in puddles)
    print(puddles)
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            coord = (i, j)
            if (i == 1 and j == 1) or coord in puddles:
                continue
            # 위쪽에서 오는 경로의 수와 왼쪽에서 오는 경로의 수를 더함
            dp[i][j] = (dp[i - 1][j] + dp[i][j - 1]) % MOD
            print(dp)
    print()
    return dp[n][m]


In [18]:
import os
import re
import shutil

In [13]:
a = "2025-01-16-programmers-(Lv2)-네트워크.md"

In [1]:
a = [1]

In [2]:
a.length

AttributeError: 'list' object has no attribute 'length'

In [28]:
file_path = "D:/blog-data/content"


def change_filename(file_path):
    for i in os.listdir(file_path):
        full_path = os.path.join(file_path, i)
        if os.path.isdir(full_path):
            change_filename(full_path)
        else:
            new_filename = re.sub(r"^\d{4}-\d{2}-\d{2}-", "", i)
            shutil.move(full_path, os.path.join(file_path, new_filename))
            print(file_path, new_filename)


change_filename(file_path)

D:/blog-data/content\Coding Test programmers-(Lv2)-네트워크.md
D:/blog-data/content\Coding Test programmers-(Lv2)-등굣길.md
D:/blog-data/content\Coding Test programmers-(Lv2)-멀리-뛰기.md
D:/blog-data/content\Coding Test programmers-(Lv2)-올바른-괄호.md
D:/blog-data/content\Coding Test Programmers-(Lv2)-최댓값과-최솟값.md
D:/blog-data/content\Coding Test programmers-(Lv2)-더-맵게.md
D:/blog-data/content\Coding Test _index.md
D:/blog-data/content\posts\AI\Prompt Engineering _index.md
D:/blog-data/content\posts\AI _index.md
D:/blog-data/content\posts\Backend\Database Systems Data-Inconsistency.md
D:/blog-data/content\posts\Backend\Database Systems\Database Optimization Cardinality.md
D:/blog-data/content\posts\Backend\Database Systems\Database Optimization Database-Indexing.md
D:/blog-data/content\posts\Backend\Database Systems\Database Optimization Database-Query-Optimization.md
D:/blog-data/content\posts\Backend\Database Systems\Database Optimization replication.md
D:/blog-data/content\posts\Backend\Database Sy

In [ ]:
solution(4, 3, [[2, 2]])

In [16]:
from collections import defaultdict, Counter


def solution(want, number, discount):
    d1 = {product: count for product, count in zip(want, number)}
    d2 = defaultdict(int)

    for i in range(10):
        d2[discount[i]] += 1
    print(d1, d2)
    answer = 0
    right = 10

    for product in discount:
        if d1 == d2:
            answer += 1
        if right < len(discount):
            d2[product] -= 1
            d2[discount[right]] += 1
            if d2[product] == 0:
                del d2[product]
        else:
            return answer
        right += 1


In [ ]:
solution(
    ["banana", "apple", "rice", "pork", "pot"],
    [3, 2, 2, 2, 1],
    [
        "chicken",
        "apple",
        "apple",
        "banana",
        "rice",
        "apple",
        "pork",
        "banana",
        "pork",
        "rice",
        "pot",
        "banana",
        "apple",
        "banana",
    ],
)


In [ ]:
#!/bin/python3

import math
import os
import random
import re
import sys


#
# Complete the 'FibonacciIterator' class below.
#
# The class is expected to have a constructor with a single argument and iterable object.
#


class FibonacciIterator:
    # Write your code here
    def __init__(self, n):
        self.n = n
        self.current = 0
        self.a, self.b = 0, 1

    def __iter__(self):
        self.current = 0
        self.a, self.b = 0, 1

    def __next__(self):
        if self.current >= self.n:
            raise StopIteration

        if self.current == 0:
            self.current += 1
            return self.a

        self.a, self.b = self.b, self.a + self.b
        return self.a


n = int(input().strip())
result = FibonacciIterator(5)
for i in result:
    print (i)

# if __name__ == "__main__":
#     fptr = open(os.environ["OUTPUT_PATH"], "w")


#     if type(result).__name__ != "FibonacciIterator":
#         fptr.write("Class is not implemented properly\n")

#     for num in result:
#         fptr.write(str(num) + "\n")

#     fptr.close()
